## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Extract clinical events and relations](https://aws.amazon.com/marketplace/pp/prodview-oi26faao26z54)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Background

- **Model**: `en.explain_doc.era`

- **Model Description**: It extracts clinical entities, assign assertion status and find temporal relationships between clinical entities.



In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "jsl-explain-doc-era"

content_type = "application/json"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

--------------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3


# Set display options
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None) 


def process_data_and_invoke_realtime_endpoint(data_dicts):
    for data_dict in data_dicts:
        json_input_data = json.dumps(data_dict)    
        i = 1
        input_file_name = f'inputs/real-time/input{i}.json'
        output_file_name = f'outputs/real-time/out{i}.out'
        
        while os.path.exists(input_file_name) or os.path.exists(output_file_name):
            i += 1
            input_file_name = f'inputs/real-time/input{i}.json'
            output_file_name = f'outputs/real-time/out{i}.out'

        os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
        os.makedirs(os.path.dirname(output_file_name), exist_ok=True)
        
        with open(input_file_name, 'w') as f:
            f.write(json_input_data)
        
        s3_client.put_object(Bucket=s3_bucket, Key=f"validation-input-json/{os.path.basename(input_file_name)}", Body=bytes(json_input_data.encode('UTF-8')))
        
        response = sm_runtime.invoke_endpoint(
            EndpointName=model_name,
            ContentType=content_type,
            Accept="application/json",
            Body=json_input_data,
        )

        # Process response
        response_data = json.loads(response["Body"].read().decode("utf-8"))
        df = pd.DataFrame(response_data['predictions'], columns=['document', 'relation', 'entity1', 'entity1_begin', 'entity1_end', 'chunk1','entity2', 'entity2_begin', 'entity2_end','chunk2', 'confidence'])
        display(df)

        # Save response data to file
        with open(output_file_name, 'w') as f_out:
            json.dump(response_data, f_out, indent=4)


## Input Format

To use the model for text prediction, you need to provide input in one of the following supported formats:

### Format 1: Array of Text Documents

Use an array containing multiple text documents. Each element represents a separate text document.

```json

{
    "text": [
        
        "She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",
        "Mike has cancer."
    ]
}
```

### Format 2: Single Text Document

Provide a single text document as a string.

```json
{
    "text": "She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge."
}
```

In [7]:
data_dicts = [ { "text" :["She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.", "Mike has cancer."]}
             ]
process_data_and_invoke_realtime_endpoint(data_dicts)


,document,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",AFTER,OCCURRENCE,7,14,admitted,CLINICAL_DEPT,19,43,The John Hopkins Hospital,0.9621104
1,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",OVERLAP,OCCURRENCE,7,14,admitted,DATE,45,54,2 days ago,0.99970835
2,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",BEFORE,OCCURRENCE,7,14,admitted,PROBLEM,74,102,gestational diabetes mellitus,0.99985516
3,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",OVERLAP,CLINICAL_DEPT,19,43,The John Hopkins Hospital,DATE,45,54,2 days ago,0.85771245
4,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",BEFORE,CLINICAL_DEPT,19,43,The John Hopkins Hospital,PROBLEM,74,102,gestational diabetes mellitus,0.90553373
5,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",OVERLAP,DATE,45,54,2 days ago,PROBLEM,74,102,gestational diabetes mellitus,0.91259193
6,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",BEFORE,EVIDENTIAL,119,124,denied,PROBLEM,126,129,pain,0.9999995
7,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",BEFORE,EVIDENTIAL,119,124,denied,PROBLEM,135,146,any headache,1.0
8,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus 

In [8]:
data_dicts = [
    {"text" : "She was admitted to Mayo Clinic three days ago following a diagnosis of Type 2 diabetes mellitus. She reported no discomfort or episodes of dizziness. The diabetes management team evaluated her, and she was released on 04/02/2018 with a prescription for 30 units of insulin detemir, 10 units of insulin aspart before meals, and metformin 850 mg twice daily. She was scheduled for regular follow-up appointments with the diabetes care team after her discharge."},
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,document,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,"She was admitted to Mayo Clinic three days ago following a diagnosis of Type 2 diabetes mellitus. She reported no discomfort or episodes of dizziness. The diabetes management team evaluated her, and she was released on 04/02/2018 with a prescription for 30 units of insulin detemir, 10 units of insulin aspart before meals, and metformin 850 mg twice daily. She was scheduled for regular follow-up appointments with the diabetes care team after her discharge.",AFTER,OCCURRENCE,8,15,admitted,CLINICAL_DEPT,20,30,Mayo Clinic,0.9999999
1,"She was admitted to Mayo Clinic three days ago following a diagnosis of Type 2 diabetes mellitus. She reported no discomfort or episodes of dizziness. The diabetes management team evaluated her, and she was released on 04/02/2018 with a prescription for 30 units of insulin detemir, 10 units of insulin aspart before meals, and metformin 850 mg twice daily. She was scheduled for regular follow-up appointments with the diabetes care team after her discharge.",OVERLAP,OCCURRENCE,8,15,admitted,DURATION,32,41,three days,0.9995034
2,"She was admitted to Mayo Clinic three days ago following a diagnosis of Type 2 diabetes mellitus. She reported no discomfort or episodes of dizziness. The diabetes management team evaluated her, and she was released on 04/02/2018 with a prescription for 30 units of insulin detemir, 10 units of insulin aspart before meals, and metformin 850 mg twice daily. She was scheduled for regular follow-up appointments with the diabetes care team after her discharge.",BEFORE,OCCURRENCE,8,15,admitted,PROBLEM,72,95,Type 2 diabetes mellitus,0.9994491
3,"She was admitted to Mayo Clinic three days ago following a diagnosis of Type 2 diabetes mellitus. She reported no discomfort or episodes of dizziness. The diabetes management team evaluated her, and she was released on 04/02/2018 with a prescription for 30 units of insulin detemir, 10 units of insulin aspart before meals, and metformin 850 mg twice daily. She was scheduled for regular follow-up appointments with the diabetes care team after her discharge.",OVERLAP,CLINICAL_DEPT,20,30,Mayo Clinic,DURATION,32,41,three days,0.53755033
4,"She was admitted to Mayo Clinic three days ago following a diagnosis of Type 2 diabetes mellitus. She reported no discomfort or episodes of dizziness. The diabetes management team evaluated her, and she was released on 04/02/2018 with a prescription for 30 units of insulin detemir, 10 units of insulin aspart before meals, and metformin 850 mg twice daily. She was scheduled for regular follow-up appointments with the diabetes care team after her discharge.",BEFORE,CLINICAL_DEPT,20,30,Mayo Clinic,PROBLEM,72,95,Type 2 diabetes mellitus,0.9999999
5,"She was admitted to Mayo Clinic three days ago following a diagnosis of Type 2 diabetes mellitus. She reported no discomfort or episodes of dizziness. The diabetes management team evaluated her, and she was released on 04/02/2018 with a prescription for 30 units of insulin detemir, 10 units of insulin aspart before meals, and metformin 850 mg twice daily. She was scheduled for regular follow-up appointments with the diabetes care team after her discharge.",OVERLAP,DURATION,32,41,three days,PROBLEM,72,95,Type 2 diabetes mellitus,0.94303304
6,"She was admitted to Mayo Clinic three days ago following a diagnosis of Type 2 diabetes mellitus. She reported no discomfort or episodes of dizziness. The diabetes management team evaluated her, and she was released on 04/02/2018 with a prescription for 30 units of insulin detemir, 10 units of insulin aspart before meals, and metformin 850 mg twice daily. She was scheduled for regular follow-up appointments with the diabetes care team after her discharge.",BEFORE,EVIDENTIAL,102,109,reported,PROBLEM,114,123,discomfort,0.99984896
7,"She was admitted to Mayo Clinic three days ago following a diagnosis of Type 2 diabetes mell

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [9]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [10]:
validation_input_path = f"s3://{s3_bucket}/validation-input-json/"
validation_output_path = f"s3://{s3_bucket}/validation-output-json/"

In [11]:
input_file_name = 'inputs/batch/input.json'
output_file_name = 'outputs/batch/out.out'

os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

In [12]:
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    accept="application/json",
)
transformer.transform(validation_input_path, content_type=content_type)
transformer.wait()

INFO:sagemaker:Creating transform job with name: jsl-explain-doc-era-2024-02-27-10-28-58-193


............................................................INFO:     Will watch for changes in these directories: ['/app']
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:     Started reloader process [7] using StatReload
24/02/27 10:39:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/27 10:39:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
#015[Stage 0:>                                                          (0 + 1) / 1]#015#015                                                                                #015#015[Stage 13:===========================================>              (3 + 1) / 4]#015#015                                                                                #015#01

In [13]:
validation_file_name = "input.json"
input_json_data = {"text" :["She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.", "Mike has cancer."]}
json_input_data = json.dumps(input_json_data)
with open(input_file_name, 'w') as f:
    f.write(json_input_data)

In [14]:
from urllib.parse import urlparse

parsed_url = urlparse(transformer.output_path)
file_key = f"{parsed_url.path[1:]}/{validation_file_name}.out"
response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

data = json.loads(response["Body"].read().decode("utf-8"))
df = pd.DataFrame(data['predictions'], columns=['document', 'relation', 'entity1', 'entity1_begin', 'entity1_end', 'chunk1','entity2', 'entity2_begin', 'entity2_end', 'chunk2','confidence'])
display(df)

with open(output_file_name, 'w') as f_out:
    json.dump(data, f_out, indent=4)

,document,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",AFTER,OCCURRENCE,7,14,admitted,CLINICAL_DEPT,19,43,The John Hopkins Hospital,0.9621104
1,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",OVERLAP,OCCURRENCE,7,14,admitted,DATE,45,54,2 days ago,0.99970835
2,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",BEFORE,OCCURRENCE,7,14,admitted,PROBLEM,74,102,gestational diabetes mellitus,0.99985516
3,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",OVERLAP,CLINICAL_DEPT,19,43,The John Hopkins Hospital,DATE,45,54,2 days ago,0.85771245
4,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",BEFORE,CLINICAL_DEPT,19,43,The John Hopkins Hospital,PROBLEM,74,102,gestational diabetes mellitus,0.90553373
5,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",OVERLAP,DATE,45,54,2 days ago,PROBLEM,74,102,gestational diabetes mellitus,0.91259193
6,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",BEFORE,EVIDENTIAL,119,124,denied,PROBLEM,126,129,pain,0.9999995
7,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge.",BEFORE,EVIDENTIAL,119,124,denied,PROBLEM,135,146,any headache,1.0
8,"She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus 

In [15]:
model.delete_model()

INFO:sagemaker:Deleting model with name: jsl-explain-doc-era-2024-02-27-10-28-55-333


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

